In [1]:
# Import standard libraries
import pandas as pd
import numpy as np
from glob import glob

# Import image processing/reading libraries
import cv2
import matplotlib.pyplot as plt
from skimage import transform
import albumentations as A

## Donnees d'applications

In [12]:
# Tables principales avec un crédit par ligne
application_test=pd.read_csv("../data/home-credit-default-risk/application_test.csv")
application_train=pd.read_csv("../data/home-credit-default-risk/application_train.csv")
full_application=pd.concat([application_train,application_test],axis=0)
full_application.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 48743
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(66), int64(40), object(16)
memory usage: 334.3+ MB


In [14]:
# Donnees concernant les credits precedents des clients dans nos bases principales, une ligne par credit precedent
bureau=pd.read_csv("../data/home-credit-default-risk/bureau.csv")
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [22]:
import swifter
# Donnees concernants les "balances" mensuelles pour chaque credits precedents
bureau_balance=pd.read_csv("../data/home-credit-default-risk/bureau_balance.csv")
bureau_balance_latest=bureau_balance[bureau_balance["MONTHS_BALANCE"]==0]
bureau=bureau.join(bureau_balance_latest,on="SK_BUREAU_ID",how="left",rsuffix="bureau_balance")
bureau_contrat=pd.DataFrame(set(bureau["SK_ID_CURR"]),columns=["SK_ID_CURR"])
bureau_contrat["distinct_loans"]=bureau_contrat["SK_ID_CURR"].swifter.apply(lambda x: len(set(bureau[bureau["SK_ID_CURR"]==x]["SK_BUREAU_ID"])))
bureau_contrat["average_nb_times_prolonged"]=bureau_contrat["SK_ID_CURR"].swifter.apply(lambda x: bureau[bureau["SK_ID_CURR"]==x]["CREDIT_DAY_OVERDUE"].mean())
bureau_contrat["sum_current_debt"]=bureau_contrat["SK_ID_CURR"].swifter.apply(lambda x: bureau[bureau["SK_ID_CURR"]==x]["AMT_CREDIT_SUM_DEBT"].sum())
bureau_contrat["nb_credits_type"]=bureau_contrat["SK_ID_CURR"].swifter.apply(lambda x: len(set(bureau[bureau["SK_ID_CURR"]==x]["CREDIT_TYPE"].mean())))
bureau_contrat["sum_overdue_debt"]=bureau_contrat["SK_ID_CURR"].swifter.apply(lambda x: len(set(bureau[bureau["SK_ID_CURR"]==x]["AMT_CREDIT_SUM_OVERDUE"].mean())))

In [ ]:
# Donnees concernants les "balances" mensuelles pour chaque credits precedents
bureau_balance=pd.read_csv("../data/home-credit-default-risk/bureau_balance.csv")
# donnees concernants les "balances" mensuelles de cartes de credit precedentes que le client avait avec home credit 
credit_card_balance=pd.read_csv("../data/home-credit-default-risk/credit_card_balance.csv")
# description des colonnnes des differents fichiers 
HomeCredit_columns_description=pd.read_csv("../data/home-credit-default-risk/HomeCredit_columns_description.csv")
# historique de repaiements de credits precedemment accordes chez home credit 
installments_payments=pd.read_csv("../data/home-credit-default-risk/installments_payments.csv")
# Donnees concernants les "balances" mensuelles des POS et credits precedents qu'un client a eu avec home credit
POS_CASH_balance=pd.read_csv("../data/home-credit-default-risk/POS_CASH_balance.csv")
# applications precedentes avec home credit de clients qui ont un credit chez eux
previous_application=pd.read_csv("../data/home-credit-default-risk/previous_application.csv")
# 
sample_submission=pd.read_csv("../data/home-credit-default-risk/sample_submission.csv")